Advanced RAG Q$A project with langchain 

In [7]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [8]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [9]:
wiki.name

'wikipedia'

In [10]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_together.embeddings import TogetherEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
load_dotenv()
# Initialize Together AI embeddings
embedding = TogetherEmbeddings(
    model='BAAI/bge-base-en-v1.5',
    together_api_key=os.getenv("TOGETHER_API_KEY")
)
loader=WebBaseLoader("https://arxiv.org/list/stat.ML/recent")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embedding=embedding)
retriever=vectordb.as_retriever()
retriever



USER_AGENT environment variable not set, consider setting it to identify your requests.


VectorStoreRetriever(tags=['FAISS', 'TogetherEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000237E2EC2390>, search_kwargs={})

In [11]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,'langsmith_search',
                      'search for information about Langsmith. for any questions about langsmith,you must use this tool')

In [12]:
retriever_tool.name

'langsmith_search'

In [ ]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name


'arxiv'

In [14]:
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\GENAI_projects\\Multisearch_agent_RAG\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='search for information about Langsmith. for any questions about langsmith,you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000237E2AE37E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'TogetherEmbeddings'], vectorstore=<langc

In [15]:
from langchain_together import ChatTogether
from langchain.schema.messages import HumanMessage
import os
from dotenv import load_dotenv
os.environ["TOGETHERAI_API"]=os.getenv("TOGETHER_API_KEY") 
llm = ChatTogether(
    model="mistralai/Mistral-7B-Instruct-v0.2",  # or any Together model
    temperature=0.7,
)


In [16]:
from langchain import hub

# Get the prompt to use - you can modify 

prompt = hub.pull("hwchase17/openai-functions-agent")
print(prompt.messages)

d:\GENAI_projects\Multisearch_agent_RAG\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')]


In [17]:
## Agents - the core idea of agents is to use a language model to choose a sequence of actions to take in chains , a sequence of actions is hardcoded (incode)

from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [18]:

# Agent Executer 
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [19]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...
 Langchain's Langsmith is a cutting-edge tool that uses advanced Language Model technology to translate text and generate human-like responses in real-time. It's designed to mimic human-like conversation, making it useful for various applications such as customer service, content creation, and more.

While Langchain's Langsmith is not specifically developed by me, I can help you understand how it works and how it can be beneficial for your needs. It's important to note that Langchain's Langsmith continuously learns and improves over time, making it more efficient and accurate in its responses.

If you have any specific questions about Langchain's Langsmith or how to use it for your purposes, feel free to ask! I'm here to help.

> Finished chain.


{'input': 'Tell me about Langsmith',
 'output': " Langchain's Langsmith is a cutting-edge tool that uses advanced Language Model technology to translate text and generate human-like responses in real-time. It's designed to mimic human-like conversation, making it useful for various applications such as customer service, content creation, and more.\n\nWhile Langchain's Langsmith is not specifically developed by me, I can help you understand how it works and how it can be beneficial for your needs. It's important to note that Langchain's Langsmith continuously learns and improves over time, making it more efficient and accurate in its responses.\n\nIf you have any specific questions about Langchain's Langsmith or how to use it for your purposes, feel free to ask! I'm here to help."}

In [20]:
agent_executor.invoke({"input":"Tell me about MachineLearning"})



> Entering new AgentExecutor chain...
 Machine Learning (ML) is a subset of artificial intelligence (AI) that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. It focuses on the development of computer programs that can access data and use it to learn for themselves.

There are three main types of machine learning:

1. Supervised Learning: The model is trained using a labeled dataset. Labeled data means the correct output is provided for each input in the dataset. Examples include linear regression, logistic regression, support vector machines, and neural networks.

2. Unsupervised Learning: The model is not provided with any labeled data. Instead, it needs to find patterns and structure within the data on its own. Examples include k-means clustering and principal component analysis (PCA).

3. Reinforcement Learning: The model learns by interacting with its environment. It gets rewards or penalties for the actions it 

{'input': 'Tell me about MachineLearning',
 'output': " Machine Learning (ML) is a subset of artificial intelligence (AI) that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. It focuses on the development of computer programs that can access data and use it to learn for themselves.\n\nThere are three main types of machine learning:\n\n1. Supervised Learning: The model is trained using a labeled dataset. Labeled data means the correct output is provided for each input in the dataset. Examples include linear regression, logistic regression, support vector machines, and neural networks.\n\n2. Unsupervised Learning: The model is not provided with any labeled data. Instead, it needs to find patterns and structure within the data on its own. Examples include k-means clustering and principal component analysis (PCA).\n\n3. Reinforcement Learning: The model learns by interacting with its environment. It gets rewards or penalt